<h1>2b. Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


Read data created in the previous chapter.

In [2]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header = None, names = CSV_COLUMNS)

<h2> Train and eval input functions to read from Pandas Dataframe </h2>

In [3]:
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [4]:
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

Our input function for predictions is the same except we don't provide a label

In [6]:
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

### Create feature columns for estimator

In [7]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

<h3> Linear Regression with tf.Estimator framework </h3>

In [8]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_train_input_fn(df_train, num_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_evaluation_master': '', '_model_dir': 'taxi_trained', '_is_chief': True, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_task_type': 'worker', '_session_config': None, '_service': None, '_task_id': 0, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc44a994da0>, '_keep_checkpoint_every_n_hours': 10000, '_train_distribute': None, '_master': '', '_global_id_in_cluster': 0}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:step = 1, loss = 212

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [11]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  print(metrics)
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))

print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-24-17:20:46
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-24-17:20:46
INFO:tensorflow:Saving dict for global step 608: average_loss = 108.883125, global_step = 608, loss = 12949.314
{'global_step': 608, 'average_loss': 108.883125, 'loss': 12949.314}
RMSE on dataset = 10.434707641601562


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [12]:
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
for items in predictions:
  print(items)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([11.384558], dtype=float32)}
{'predictions': array([11.381897], dtype=float32)}
{'predictions': array([11.382797], dtype=float32)}
{'predictions': array([11.380239], dtype=float32)}
{'predictions': array([11.384284], dtype=float32)}
{'predictions': array([11.384054], dtype=float32)}
{'predictions': array([11.382571], dtype=float32)}
{'predictions': array([11.382606], dtype=float32)}
{'predictions': array([11.384446], dtype=float32)}
{'predictions': array([11.382201], dtype=float32)}
{'predictions': array([11.384559], dtype=float32)}
{'predictions': array([11.384731], dtype=float32)}
{'predictions': array([11.37771], dtype=float32)}
{'predictions': array([11.381803], dtype=float32)}
{'predictions': array([1

{'predictions': array([11.380982], dtype=float32)}
{'predictions': array([11.383216], dtype=float32)}
{'predictions': array([11.621212], dtype=float32)}
{'predictions': array([11.381415], dtype=float32)}
{'predictions': array([11.383719], dtype=float32)}
{'predictions': array([11.441811], dtype=float32)}
{'predictions': array([11.442604], dtype=float32)}
{'predictions': array([11.499933], dtype=float32)}
{'predictions': array([11.382625], dtype=float32)}
{'predictions': array([11.620614], dtype=float32)}
{'predictions': array([11.381604], dtype=float32)}
{'predictions': array([11.500628], dtype=float32)}
{'predictions': array([11.377107], dtype=float32)}
{'predictions': array([11.37437], dtype=float32)}
{'predictions': array([11.380119], dtype=float32)}
{'predictions': array([11.381839], dtype=float32)}
{'predictions': array([11.36368], dtype=float32)}
{'predictions': array([11.382122], dtype=float32)}
{'predictions': array([11.367124], dtype=float32)}
{'predictions': array([11.441227]

{'predictions': array([11.38283], dtype=float32)}
{'predictions': array([11.437558], dtype=float32)}
{'predictions': array([11.380777], dtype=float32)}
{'predictions': array([11.381911], dtype=float32)}
{'predictions': array([11.381724], dtype=float32)}
{'predictions': array([11.378086], dtype=float32)}
{'predictions': array([11.382255], dtype=float32)}
{'predictions': array([11.383416], dtype=float32)}
{'predictions': array([11.382811], dtype=float32)}
{'predictions': array([11.441819], dtype=float32)}
{'predictions': array([11.621047], dtype=float32)}
{'predictions': array([11.382223], dtype=float32)}
{'predictions': array([11.382471], dtype=float32)}
{'predictions': array([11.622472], dtype=float32)}
{'predictions': array([11.382254], dtype=float32)}
{'predictions': array([11.620898], dtype=float32)}
{'predictions': array([11.382904], dtype=float32)}
{'predictions': array([11.381772], dtype=float32)}
{'predictions': array([11.618112], dtype=float32)}
{'predictions': array([11.678359

{'predictions': array([11.377391], dtype=float32)}
{'predictions': array([11.382089], dtype=float32)}
{'predictions': array([11.378074], dtype=float32)}
{'predictions': array([11.437009], dtype=float32)}
{'predictions': array([11.384584], dtype=float32)}
{'predictions': array([11.381305], dtype=float32)}
{'predictions': array([11.382421], dtype=float32)}
{'predictions': array([11.443351], dtype=float32)}
{'predictions': array([11.429336], dtype=float32)}
{'predictions': array([11.435217], dtype=float32)}
{'predictions': array([11.383129], dtype=float32)}
{'predictions': array([11.442666], dtype=float32)}
{'predictions': array([11.383531], dtype=float32)}
{'predictions': array([11.383018], dtype=float32)}
{'predictions': array([11.441015], dtype=float32)}
{'predictions': array([11.384599], dtype=float32)}
{'predictions': array([11.382159], dtype=float32)}
{'predictions': array([11.383019], dtype=float32)}
{'predictions': array([11.382049], dtype=float32)}
{'predictions': array([11.37602

{'predictions': array([11.3821945], dtype=float32)}
{'predictions': array([11.3828745], dtype=float32)}
{'predictions': array([11.382655], dtype=float32)}
{'predictions': array([11.442504], dtype=float32)}
{'predictions': array([11.441861], dtype=float32)}
{'predictions': array([11.382388], dtype=float32)}
{'predictions': array([11.502536], dtype=float32)}
{'predictions': array([11.500923], dtype=float32)}
{'predictions': array([11.379973], dtype=float32)}
{'predictions': array([11.38433], dtype=float32)}
{'predictions': array([11.38182], dtype=float32)}
{'predictions': array([11.439924], dtype=float32)}
{'predictions': array([11.383777], dtype=float32)}
{'predictions': array([11.381753], dtype=float32)}
{'predictions': array([11.445236], dtype=float32)}
{'predictions': array([11.441662], dtype=float32)}
{'predictions': array([11.620217], dtype=float32)}
{'predictions': array([11.382692], dtype=float32)}
{'predictions': array([11.382485], dtype=float32)}
{'predictions': array([11.38384

{'predictions': array([11.4446535], dtype=float32)}
{'predictions': array([11.353049], dtype=float32)}
{'predictions': array([11.382444], dtype=float32)}
{'predictions': array([11.380124], dtype=float32)}
{'predictions': array([11.385787], dtype=float32)}
{'predictions': array([11.382213], dtype=float32)}
{'predictions': array([11.383776], dtype=float32)}
{'predictions': array([11.384071], dtype=float32)}
{'predictions': array([11.381841], dtype=float32)}
{'predictions': array([11.619248], dtype=float32)}
{'predictions': array([11.381226], dtype=float32)}
{'predictions': array([11.557243], dtype=float32)}
{'predictions': array([11.38524], dtype=float32)}
{'predictions': array([11.38298], dtype=float32)}
{'predictions': array([11.381769], dtype=float32)}
{'predictions': array([11.381587], dtype=float32)}
{'predictions': array([11.678824], dtype=float32)}
{'predictions': array([11.36262], dtype=float32)}
{'predictions': array([11.384427], dtype=float32)}
{'predictions': array([11.619889]

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [13]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 100));
print_rmse(model, df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_evaluation_master': '', '_model_dir': 'taxi_trained', '_is_chief': True, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_task_type': 'worker', '_session_config': None, '_service': None, '_task_id': 0, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc42a123470>, '_keep_checkpoint_every_n_hours': 10000, '_train_distribute': None, '_master': '', '_global_id_in_cluster': 0}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:step = 1, loss = 346

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [ ]:
import google.datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  EXTRACT(DAYOFWEEK FROM pickup_datetime) * 1.0 AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) * 1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  CONCAT(CAST(pickup_datetime AS STRING), CAST(pickup_longitude AS STRING), CAST(pickup_latitude AS STRING), CAST(dropoff_latitude AS STRING), CAST(dropoff_longitude AS STRING)) AS key
FROM
  `nyc-tlc.yellow.trips`
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) = {1}".format(base_query, phase)
  else:
    query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), {1}) = {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).execute().result().to_dataframe()

In [ ]:
print_rmse(model, df)

RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

In [1]:
import random 
import tensorflow as tf
import shutil

def calc_volume(r, h):
  return 2*3.14*r*r*h

feature_cols = [tf.feature_column.numeric_column("radius"), tf.feature_column.numeric_column("height")];

def train_input_factory(limit):
  def train_input_fn():
    r = [0.2 + 0.01 * random.randint(1, 180) for i in range(limit)]
    h = [0.2 + 0.01 * random.randint(1, 180) for i in range(limit)]
    v = [ calc_volume(r1, h1) for r1, h1 in zip(r, h) ]

    features = { "radius": r, "height": h}
    return features, v
  return train_input_fn

shutil.rmtree("DNNModel")
model = tf.estimator.DNNRegressor(feature_columns = feature_cols, 
                                   hidden_units = [6,6,3,2],
                                 model_dir = "DNNModel")

model.train(train_input_factory(1000), steps=15000);


/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_task_id': 0, '_session_config': None, '_save_checkpoints_steps': None, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f66ca71fd30>, '_keep_checkpoint_max': 5, '_model_dir': 'DNNModel', '_num_worker_replicas': 1, '_train_distribute': None, '_service': None, '_task_type': 'worker', '_evaluation_master': '', '_tf_random_seed': None, '_num_ps_replicas': 0}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into DNNModel/model.ckpt.
INFO:tensorflow:step = 1, loss = 210090.12
I

INFO:tensorflow:global_step/sec: 844.975
INFO:tensorflow:step = 7401, loss = 182.64929 (0.118 sec)
INFO:tensorflow:global_step/sec: 870.056
INFO:tensorflow:step = 7501, loss = 180.84402 (0.116 sec)
INFO:tensorflow:global_step/sec: 838.415
INFO:tensorflow:step = 7601, loss = 179.66022 (0.119 sec)
INFO:tensorflow:global_step/sec: 591.347
INFO:tensorflow:step = 7701, loss = 178.69595 (0.169 sec)
INFO:tensorflow:global_step/sec: 813.403
INFO:tensorflow:step = 7801, loss = 177.82036 (0.124 sec)
INFO:tensorflow:global_step/sec: 885.317
INFO:tensorflow:step = 7901, loss = 177.03061 (0.112 sec)
INFO:tensorflow:global_step/sec: 808.519
INFO:tensorflow:step = 8001, loss = 176.22232 (0.124 sec)
INFO:tensorflow:global_step/sec: 854.836
INFO:tensorflow:step = 8101, loss = 175.53026 (0.117 sec)
INFO:tensorflow:global_step/sec: 853.788
INFO:tensorflow:step = 8201, loss = 174.80467 (0.117 sec)
INFO:tensorflow:global_step/sec: 920.128
INFO:tensorflow:step = 8301, loss = 173.7829 (0.109 sec)
INFO:tensor

In [2]:
import numpy as np

metrics = model.evaluate(input_fn = train_input_factory(100), steps=1);
print(metrics)
print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-24-18:24:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from DNNModel/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2019-04-24-18:24:34
INFO:tensorflow:Saving dict for global step 15000: average_loss = 0.0682014, global_step = 15000, loss = 6.82014
{'loss': 6.82014, 'average_loss': 0.0682014, 'global_step': 15000}
RMSE on dataset = 0.2611539661884308


Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License